<a href="https://colab.research.google.com/github/zeyuanyin/NLP702/blob/main/tutorial/text_classification_attention/rnn_attention_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recurrent Neural Networks with Attention

### Goal of this tutorial
- Introduce Recurrent Neural Networks (RNNs)
- Implement RNN for sentiment analysis
- Combine RNN and Self-attention
- Implement Long-Short Term Memories (LSTMs) for sentiment analysis
- Implement Gated Recurrent Units (GRUs) for sentiment analysis

### General
- This notebook was last tested on Python 3.7, PyTorch 1.8.0, torchtext 0.5.0

We would like to acknowledge the following materials which helped as a reference in preparing this tutorial:
- https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf

In [13]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/course/nlp702
!pip install torchtext==0.5.0

Mounted at /content/drive
/content/drive/MyDrive/course/nlp702
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

### Embedding Layer

The ``Embedding`` layer in Pytorch is where we pass our vocabulary to get get back a word vector for each word in the vocabulary. We need to know what the input and output of this layer look like. Let's first
do this on a dummy example where we have two sentences ``x_1`` and ``x_2``. Let's assume we have the two sentences as:

In [3]:
x_1 = "He is very nice"
x_2 = "She is very kind"

Let's convert the two sentences into indexes (each word is replaced with its index in the vocabulary).
Let's assume our ``vocabulary size`` is set to 100. Remember, vocabulary size is a hyper-parameter.
Let's also store that ``vocabulary size`` in a variable ``VOCAB_SIZE`` now as we will need to pass it to the ``Embedding`` layer later.

In [4]:
x_1 = [1, 25, 40, 5]
x_2 = [4, 25, 40, 99]
VOCAB_SIZE = 100

### Max sequence length

One last thing we need to think about is the ``length of each sequence``. The two examples above are nicely set to equal length = 4. This does not need to be the case, as we can have sequences of varying lengths. We will be passing a batch of sentences to Pytorch and the max sequence length will be set to the length of the longest sentence in that batch. The rest of sentences (shorter ones) will be padded with zeros. Now, do we need to explicitly provide the max sequence length to Pytorch? And how do we know the max seq length for each batch, if different batches have sequences of varying lengths and each batch is set to the max sent in that batch? Well, rest assured, we don't really need to worry about that. Pytorch will assign a max seq length for each batch. We will be able to inspect the max seq length for a given batch using output of the ``Embedding`` layer. (We will see that soon).

## Size of word vector

The ``Embedding`` layer will give us a vector for each word in the vocabulary.
Now, we will need to tell it what size we want for that vector. Popular values for a vector size are usally between 100-300 for many tasks (e.g., sentiment analysis"). Let's set it to 300 dimensions. (You are encouraged to play with this value as practice). All words in the vocabulary will have the same embedding size. Let's put that hyper-parameter in a variable ``WORD_VEC_SIZE``:

In [5]:
WORD_VEC_SIZE= 300

We are now ready to call the ``Embedding`` class to construct an embeddings tensor:

In [6]:
# Constructing an embedding Layer:
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE)
input = torch.LongTensor([ x_1, x_2 ])
embedded=embedding(input)
print(embedded)
print(embedded.shape) # batch size, sequence lenght, embedding size

tensor([[[-0.0473, -1.7201, -1.0530,  ..., -1.6113,  0.6682, -0.7543],
         [-0.6174,  1.4342, -1.7113,  ...,  0.1957, -0.9740, -0.5939],
         [-0.8626,  0.1138, -0.7820,  ..., -1.4221, -0.6571, -1.8813],
         [-1.2141,  0.8867, -0.0886,  ..., -0.4370,  0.7159, -0.9371]],

        [[-0.8337,  0.1721, -1.0398,  ..., -0.5583, -0.4534, -2.5654],
         [-0.6174,  1.4342, -1.7113,  ...,  0.1957, -0.9740, -0.5939],
         [-0.8626,  0.1138, -0.7820,  ..., -1.4221, -0.6571, -1.8813],
         [ 0.3074, -0.1321, -2.4767,  ...,  0.7688, -0.1164, -0.6220]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 4, 300])


**Let's print the shape of this tensor:**

In [7]:
print(embedded.size())

torch.Size([2, 4, 300])


This is telling us:
- We have ``2 examples`` (that is, our ``x_1`` and ``x_2``). (Note: We will be paqssing a whole batch to the ``Embedding`` class and so this first dimension will be equal to the ``batch size``.
- For each of the two examples, we have a ``max sequence length`` = 4 (x_1 and x_2 each had 4 indexes).
- The ``word vector dimension`` is set to 300.

### Max sequence length: Another note

Recall from above we mentioned Pytorch automatically infers the max sequence length for each batch. 
For the example above (as you can see from the second dimension returned by ``embedded.size()``, Pytorch 
inferred the max seq length for this batch of two sentences is 4.
Let's just adjust the second example, **adding two more words** (the string "and kind"). Note, both our ``VOCAB_SIZE`` and ``WORD_VEC_SIZE`` stay the same as before. We assign the word "and" an index of "7" and the word "considerate" and index of "60". Note that we have to pad the first example ``x_1`` with zeros in the end. (Try removing the zero padding. What do you observe when you run your code with the ``Embedding`` class? Hint: You will get an error.):

In [8]:
x_1 = "He is very nice"
x_2 = "She is very kind and considerate"
x_1 = [1, 25, 40, 5, 0, 0]
x_2 = [4, 25, 40, 99, 7, 60]

Now, let's create a new ``embedding`` layer by creating a new instance of the ``Embedding`` class:

In [9]:
# Constructing an embedding Layer:
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE)
input = torch.LongTensor([x_1, x_2 ])
embedded=embedding(input)
print(embedded)

tensor([[[ 7.3561e-01,  7.0300e-01,  1.3671e+00,  ...,  1.9209e-01,
           6.0257e-01,  5.7987e-01],
         [-4.3380e-01, -8.6854e-01,  4.5886e-02,  ...,  1.0800e+00,
          -3.2959e-01, -3.0174e-01],
         [ 2.2750e+00,  1.0661e+00, -3.9394e-01,  ..., -2.4601e-01,
          -1.5952e+00,  3.3553e-02],
         [ 1.2953e+00,  1.4372e+00,  2.3082e-01,  ..., -5.9843e-01,
           1.6303e+00, -5.4260e-01],
         [ 2.3311e-01, -9.3737e-01, -1.3947e+00,  ...,  2.7655e+00,
          -9.0550e-02, -1.4447e+00],
         [ 2.3311e-01, -9.3737e-01, -1.3947e+00,  ...,  2.7655e+00,
          -9.0550e-02, -1.4447e+00]],

        [[ 6.8891e-02, -5.2670e-01,  1.3103e+00,  ...,  3.7234e-01,
           3.7740e-01,  3.9070e-01],
         [-4.3380e-01, -8.6854e-01,  4.5886e-02,  ...,  1.0800e+00,
          -3.2959e-01, -3.0174e-01],
         [ 2.2750e+00,  1.0661e+00, -3.9394e-01,  ..., -2.4601e-01,
          -1.5952e+00,  3.3553e-02],
         [-2.0625e-01,  2.6845e-03, -7.3943e-01,  ...

**If we inspect the shape of the new tensor ``embedded``, we will see the second dimension now changed to 6, to match the max sequence length:**

In [10]:
print(embedded.size())

torch.Size([2, 6, 300])


### How does Pytorch initialize word vector dimensions/weights?

Note that Pytorch initializes the word vectors with initialized from a **normal distribution** $ \mathcal{N}(0, 1) $. The word embedding weights are by default learnable parameters in Pytorch and so they will be adjusted during training. (Note: These weights can be initialized from an external word embedding tool such as Word2vec, Fasttext, or Glove. Also, the weights can be frozen, which is a reasonable option when initialized from an external tool. You can choose to keep learning them within the model with your training data). Below we show the ones initialized from a normal distribution by Pytorch.

In [11]:
embedding.weight

Parameter containing:
tensor([[ 2.3311e-01, -9.3737e-01, -1.3947e+00,  ...,  2.7655e+00,
         -9.0550e-02, -1.4447e+00],
        [ 7.3561e-01,  7.0300e-01,  1.3671e+00,  ...,  1.9209e-01,
          6.0257e-01,  5.7987e-01],
        [ 2.0081e-01, -8.3376e-01,  1.2098e+00,  ..., -1.4564e-01,
         -7.1152e-03,  4.0032e-01],
        ...,
        [ 1.9941e+00,  2.0688e-02,  1.6576e+00,  ..., -1.0220e+00,
         -6.2974e-01,  6.5592e-01],
        [-3.3078e-01,  1.2312e+00, -5.4520e-01,  ...,  6.5904e-01,
         -8.6337e-01, -1.7219e+00],
        [-2.0625e-01,  2.6845e-03, -7.3943e-01,  ...,  1.5054e+00,
         -1.2996e+00,  5.6773e-01]], requires_grad=True)

More information about the ``Embedding`` class can be found [here](https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html#Embedding).

## Recurrent Neural Networks
Recurrent Neural Networks (RNNs) are used to model sequences of arbitrary length (e.g., sequence of words in a sentence, sequence of sentences in a document, sequence of frames in a video). RNNs typically use their internal state (memory) to process sequence of inputs. At each time-step, RNNs output a prediction and hidden state, feeding its previous hidden state into each next step. RNNs are applied in a wide range of NLP applications:
- language modeling, where RNN can condition on **all** previous words in the corpus unlike n-gram language model
- text classification, where the states act as features (we will see sentiment analysis in this tutorial)
- machine translation, where a RNN is used to process a sentence in source language and another RNN is used to decode the sentence in target language.
- sequence labeling, where the states in RNN are used to predict a category for each item in the sequence.

Recommended reading for understanding the theory of RNNs: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 


### Grabbing few tweets using torchtext

Let us follow **torchtext** tutorial to read few tweets from the [sentiment analysis dataset](http://alt.qcri.org/semeval2016/task4/) used in the previous tutorial on feedforward neural networks. The preprocessed (tokenization, removing URLs, mentions, hashtags and so on) tweets are placed under ``data/sentiment-twitter-2016-task4`` folder in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``.  

Let us view few tweets from ``train.tsv`` using pandas.

In [15]:
import pandas as pd
df = pd.read_csv("./data/sentiment-twitter-2016-task4/train.tsv", sep = '\t', header=None, names=['tweet','label']) # the separator of tsv file is `\t`
df.head()

,tweet,label
0,dear <<<MENTION>>> the newooffice for mac is g...,2
1,<<<MENTION>>> how about you make a system that...,2
2,i may be ignorant on this issue but should we ...,2
3,thanks to <<<MENTION>>> i just may be switchin...,2
4,if i make a game as a <<<HASHTAG>>> universal ...,0


**We import the relevant packages, define the tokenizer and TorchText's fields.**

In [16]:
# import related packages
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset

# define the white space tokenizer to get tokens
def tokenize_en(tweet):
    """
    Tokenizes English tweet from a string into a list of strings (tokens)
    """
    return tweet.strip().split()

# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

**To use the different splits (training, development and testing), we use `TabularDataset` class to load datasets.**

In [17]:
train, val, test = TabularDataset.splits(
    path="./data/sentiment-twitter-2016-task4/", # the root directory where the data lies
    train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
    format='tsv',
    skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=[('tweet', TEXT), ('label', LABEL)])

**Build our vocabulary to map words to integers.**

In [18]:
TEXT.build_vocab(train, min_freq=3) # builds vocabulary based on all the words that occur at least twice in the training set
LABEL.build_vocab(train)

**Initialize the iterators for the train, validation, and test data. Note that we set ``sort`` as False so as to not sort examples based on similar lengths which minimizes padding in this example.**

In [19]:
from torchtext.data import Iterator, BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=False,
# An argument to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=False
)

**Create a batch of four examples and print them**

In [20]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  #we use first batch as an example.

# print the four examples with padding 
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tmp = []
    for i in range(tweets.shape[0]): # token loop
        tmp.append(TEXT.vocab.itos[tweets[i,j]])
    print(j," sample:",tmp)

processed tweets: 
0  sample: ['these', 'were', 'the', 'best', "sunday's", 'watching', 'harry', 'potter', 'with', 'my', 'girl', '<unk>', '<<<hashtag>>>', '<<<hashtag>>>', '<<<url>>>', '<<<url>>>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
1  sample: ['<unk>', '<unk>', 'will', 'have', 'played', 'for', 'psg', 'juventus', 'and', 'bayern', 'by', 'the', 'time', 'he', 'is', '<<<digit>>>', 'but', 'may', 'not', 'have', 'made', 'his', 'international', 'debut', 'some', 'feat']
2  sample: ['<unk>', 'part', 'of', 'the', 'migration', 'crisis', 'is', 'caused', 'by', 'turkey', '<unk>', 'europe', 'for', 'the', 'cold', '<unk>', 'given', 'to', 'erdogan', '<<<url>>>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
3  sample: ['my', 'weekend', 'plans', 'play', '<unk>', '<<<digit>>>', 'see', 'magic', 'mike', 'xxl', 'with', 'my', 'sister', 'on', 'sunday', 'what', 'are', 'everyone', '<unk>', 'plans', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


**Let us pick up 4 tweets from the training set and convert them to tensors.**

**Create a batch of four examples and print them**

In [21]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  #we use first batch as an example.

# print the four examples with padding 
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tmp = []
    for i in range(tweets.shape[0]): # token loop
        tmp.append(TEXT.vocab.itos[tweets[i,j]])
    print(j," sample:",tmp)

processed tweets: 
0  sample: ['the', 'new', 'iphone', 'gets', 'announced', 'today', "that's", 'really', 'all', 'that', 'matters', 'happy', 'wednesday', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
1  sample: ['<unk>', 'tomorrow', 'and', "that's", 'it', 'on', 'the', 'arsenal', 'front', '<unk>', 'to', 'watch', 'england', 'spain', 'much', 'more', '<unk>', 'to', 'watch', '<pad>', '<pad>', '<pad>', '<pad>']
2  sample: ['fleetwood', 'mac', 'are', 'responsible', 'for', 'some', 'of', 'the', 'best', 'songs', 'ever', 'recorded', 'come', 'see', 'them', 'performed', 'this', 'saturday', 'by', 'the', 'world', 'turning', '<unk>']
3  sample: ['update', 'on', 'the', 'event', 'for', 'saturday', 'sept', 'for', 'those', 'who', 'were', 'expecting', 'a', 'fleetwood', 'mac', 'open', '<unk>', 'night', '<<<url>>>', '<pad>', '<pad>', '<pad>', '<pad>']


### Creating a single hidden layer RNN

PyTorch has ``torch.nn.RNN`` module that implements the vanilla (Elman) RNN with *tanh* or *ReLU* non-linearity. The documentation for this module is [here](https://pytorch.org/docs/stable/nn.html#torch.nn.RNN). Let us use the sample batch of five examples created before to understand this module.

In this tutorial, we will represent the input tweet using a sequence of word embeddings (for each word present in the tweet). We will use ``torch.nn.Embedding module`` to store word vectors corresponding to words in the vocabulary.

Before implementing the embedding module for our usecase, let us compute the size of the word vocabulary.

In [22]:
VOCAB_SIZE = len(TEXT.vocab.stoi)
print(VOCAB_SIZE)

3343


Let us implement the embedding module (whose underlying weight matrix shape is (``vocabulary size`` $\times$ ``word embedding size``) for our usecase:

In [23]:
# an Embedding module containing 10 dimensional tensor for each word in the vocabulary
import torch
import torch.nn as nn
WORD_VEC_SIZE=300
# Note, the parameters to Embedding class below are:
# num_embeddings (int): size of the dictionary of embeddings
# embedding_dim (int): the size of each embedding vector
# For more details on Embedding class, see: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/sparse.py
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, sparse=True)

Let us now feed the tensors of our sample batch to the embedding module and extract the sequence of word embeddings for each tweet.

In [24]:
# print tensor containing word ids for our batch
print("*"*50, "\n Word ids for the first batch (recall, it has 4 sentences, each column representing a sentence): \n", tweets.data, "\n","*"*50,)

# feed the "word ids" tensor to the embedding module
tweet_input_embeddings = embedding(tweets)

# print the dimensions of the tweet_embeddings
print("*"*50, "\n Tweet input word embeddings size: ", tweet_input_embeddings.size(), "\n","*"*50,) 
# first dimension - sequence length: number of words per example (same across the whole batch, after padding) --> max_seq = 22
# second dimension -  batch size / number of examples in the batch --> 4
# third dimension - number of dimensions in the word vector

************************************************** 
 Word ids for the first batch (recall, it has 4 sentences, each column representing a sentence): 
 tensor([[   2,    0,  174,  393],
        [  49,   21,  177,    8],
        [  88,    9,   53,    2],
        [ 541,  281, 2489,  151],
        [ 490,   20,   15,   15],
        [  77,    8,   95,   67],
        [ 281,    2,   12,  256],
        [ 169,  161,    2,   15],
        [  39,  878,   91,  408],
        [  25,    0,  485,  100],
        [2993,    3,  244,  286],
        [ 175,   56, 2480, 1890],
        [ 133,  871,  121,    7],
        [   1, 1738,   41,  174],
        [   1,  225,  238,  177],
        [   1,   94, 1345,  295],
        [   1,    0,   27,    0],
        [   1,    3,   67,   59],
        [   1,   56,   66,    5],
        [   1,    1,    2,    1],
        [   1,    1,   79,    1],
        [   1,    1, 1383,    1],
        [   1,    1,    0,    1]]) 
 **************************************************
*************

Let's actually view the actual word embeddings tensor for this batch:

In [25]:
print("*"*50, "\n Embeddings for the first batch: \n", tweet_input_embeddings, "\n","*"*50,) 

************************************************** 
 Embeddings for the first batch: 
 tensor([[[ 2.1218,  0.9743, -0.1468,  ..., -0.5750,  0.6821, -0.8944],
         [ 0.7494, -1.1060, -1.2781,  ...,  0.8678, -0.6486, -0.0595],
         [-0.3703,  0.6844, -0.3862,  ..., -0.1021, -0.3727, -1.3854],
         [ 0.8089,  0.7152,  1.5386,  ..., -0.1323,  1.1486,  0.7147]],

        [[ 1.1508, -0.1211, -1.4412,  ...,  0.3643, -1.1876, -1.1646],
         [ 0.0584,  0.1019,  0.4871,  ..., -0.1036,  0.6846,  2.9003],
         [ 1.3016, -1.0430,  0.5538,  ...,  0.5468, -0.3526,  1.1384],
         [ 0.0396, -0.0064,  0.0487,  ..., -1.3168,  2.2326, -0.8042]],

        [[-0.7244,  0.2086, -0.9477,  ..., -1.2408,  1.4763, -0.1403],
         [ 0.5514, -0.6204, -3.3181,  ..., -1.5047,  0.2029, -0.6665],
         [-0.7907, -1.1186, -0.4532,  ..., -0.7396,  0.7914,  0.2314],
         [ 2.1218,  0.9743, -0.1468,  ..., -0.5750,  0.6821, -0.8944]],

        ...,

        [[-1.1722, -0.2612,  2.4188,  ...

What we are seeing is the actual word vectors representing each of the 4 sentences (i.e., whole batch).
This is dimension 2 in ``tweet_input_embeddings``. 

In [26]:
tweet_input_embeddings.size()[1]

4

As mentioned, the dimension 1 (second dimension) in ``tweet_input_embeddings`` is ``max_seq length`` in this batch:

In [27]:
tweet_input_embeddings.size()[0]

23

Now, dimension 3 in ``tweet_input_embeddings`` (indexed as 2) is the size of the word vectors:

In [28]:
tweet_input_embeddings.size()[2]

300

Let's look at the vector for the ``first word`` in the ``first sentence`` in the batch:

In [29]:
tweet_input_embeddings[:1, :1, :].shape

torch.Size([1, 1, 300])

In [30]:
tweet_input_embeddings[:1, :1, :]

tensor([[[ 2.1218,  0.9743, -0.1468, -1.2003,  1.8816, -2.4530, -1.2286,
          -0.4960,  0.4330,  0.5830,  0.1121,  1.8909, -0.5137,  0.8890,
           0.1217, -0.5173,  0.4364, -0.4072,  0.5998,  0.9329, -0.0667,
          -0.6496,  1.5419,  1.5144,  1.6667,  0.5024, -0.0499, -1.0936,
           1.1748,  0.2787,  0.9768, -0.2663,  0.5993,  0.1757,  0.4547,
          -0.4720,  0.2700,  1.0112,  0.6929,  0.0404,  0.9979,  1.0840,
          -1.4953,  0.7288, -0.7243, -1.2792,  0.2762, -0.3202, -0.4962,
          -0.2118,  1.1177,  0.1352, -0.3276,  0.0089,  1.8404,  0.5544,
           0.5006,  1.5396,  0.8025, -0.3131, -1.2787, -1.2829, -0.9398,
          -0.6095, -0.3534,  0.8061, -1.1868, -0.1298,  0.3473,  0.4607,
          -0.7264, -0.2079, -0.1168,  1.0986,  1.5933, -0.0576,  0.0211,
           0.7053,  0.2489, -0.6605,  0.8973,  0.0802, -0.1569, -0.6637,
           0.5189, -0.1655,  0.1572,  0.4629, -0.3198,  0.1901,  0.8565,
          -0.0860, -0.3288, -0.0321,  0.4358,  0.49

Let's look at the ``first 5 dimensions`` of that same ``first word`` of the ``first sentence``:

In [31]:
tweet_input_embeddings[:1, :1, :5]

tensor([[[ 2.1218,  0.9743, -0.1468, -1.2003,  1.8816]]],
       grad_fn=<SliceBackward0>)

The following shows you the ``first 5 dimensions`` of the ``first word`` from ``each of the 4 sentences``

In [32]:
tweet_input_embeddings[:1, :, :5]

tensor([[[ 2.1218,  0.9743, -0.1468, -1.2003,  1.8816],
         [ 0.7494, -1.1060, -1.2781,  0.2435,  2.0851],
         [-0.3703,  0.6844, -0.3862, -0.3838,  1.6359],
         [ 0.8089,  0.7152,  1.5386,  1.5421, -0.4786]]],
       grad_fn=<SliceBackward0>)

The following shows you the ``last 7 dimensions`` of the ``last word`` from ``the last sentence``. Enjoy!

In [33]:
tweet_input_embeddings[-1:, 3:, -7:]

tensor([[[-1.8665, -0.7544,  0.2942, -2.0007,  0.1687, -0.2265, -1.1488]]],
       grad_fn=<SliceBackward0>)

We will be passing the embedding vector for the first word, Simultaneously for each senetence in the batch, to the RNN. But let's now define an RNN module first:

In [34]:
"""
define the RNN module
"""
# first input - number of dimesnions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden state h_t (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 1)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=1) # input_size, hidden_size, num_layers

We will now pass the ``tweet_input_embeddings`` (representations of words in our batch) to RNN. Before we do, we need to know RNN also *optionally* takes a parameter for the ``initial hidden state h0`` (that is, the hidden state we will input to the model before it runs. It is just something that we need to start the model. If we don't provide it, Pytorch will just initialize h0 to a tensor of zeros. 

Let's construct an ``initial hidden state h0``. Note the shape of its tensor, and what each of the 3 parameters it takes mean.

In [35]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (1)
# second dimension - number of examples/sentences in a batch
# third dimension - number of nodes in hidden layer (50, size of the hidden layer)
h0 = torch.randn(1, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([1, 4, 50])


Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.
We will get back two objects ``output`` and ``hn`` that we will need to understand.

In [36]:
"""
forward propagation over the RNN model
"""
output, hn = rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's of apprpriate size (num_layers, batch, hidden_size) when not provided

But what is ``output``? Well, let's inspect its shape first:

In [37]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([23, 4, 50])


Here's what we need to know about ``output``:
- The first dimension in the ``output`` tensor is the ``max_seq length``. 
- The second dimension is ``batch_size`` (the number of examples/sentences in our batch = 4).
- The third dimension is the ``size of nodes/units`` in our hidden layer (=50). 

Here's what we need to know about ``hn``:
- ``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of hidden layer nodes) containing the hidden state for the last ``time step`` 
(``t = max_seq_length``). Actually, `output` is tensor containing the output features (h_t) from the layer of RNN, for each t. Namely, `output` returns all the hidden states of all time steps from RNN. Hence, the last element of `output` (i.e., `output[-1, :, :]`) is `h_n`. 

In [38]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([1, 4, 50])


You can take the output representation for a tweet after processing the last token (t=seq_len or last timestep) and call the resulting representation as the tweet representation that **"summarizes" the information present** in the tweet. This tweet representation can further be used for a useful task like tweet classification (we will try out sentiment analysis later in this tutorial) by adding a classification module on top of the tweet representation.

Let us compute the final tweet representation:

In [39]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


### Self Attention Mechanism


The fixed-length context vector carries the burden of encoding the complete "meaning" of the input sequence, regardless of its length. This causes information bottleneck problem. Attention provides a solution to the bottleneck problem.

In [40]:
print(tweet_output_embeddings.size())
print(output.size())

torch.Size([4, 50])
torch.Size([23, 4, 50])


In [41]:
batch_size = output.shape[1]
src_len = output.shape[0]

In [42]:
key_a = nn.Linear(50, 50)
q_a = nn.Linear(50, 50)

In [43]:
output_keys = key_a(output)
tweet_output_embeddings = q_a(tweet_output_embeddings)

Calculate importances between sequence-level representation and token representations.

In [44]:
attn_weights = torch.bmm(output_keys.permute(1,0,2), tweet_output_embeddings.unsqueeze(2)).squeeze(2).permute(1,0)
print(attn_weights.shape)

torch.Size([23, 4])


Ignore "PAD" tokens.

In [45]:
attn_weights = attn_weights.masked_fill(tweets == TEXT.vocab.stoi[TEXT.pad_token], -1e9)

In [46]:
soft_attn_weights = F.softmax(attn_weights, 0).permute(1,0)

In [47]:
soft_attn_weights.shape

torch.Size([4, 23])

In [48]:
new_tweet_output_embedding = torch.bmm(output.permute(1, 2, 0), soft_attn_weights.unsqueeze(2)).squeeze(2)

In [49]:
print(new_tweet_output_embedding.shape)

torch.Size([4, 50])


## Multilayered RNN

For some applications, we may need more than one hidden layer for RNN to model the information flow. Adding more layers requires fews changes.

Firstly, we change the ``num_layers`` argument to reflect the number of layers we want during the RNN module definition.

In [50]:
"""
define the RNN module
"""
# first input - number of dimesnions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 1)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to single layered RNN, Multilayered RNN module takes two inputs: the ``initial hidden state h0`` for each element in the batch (at ``time step t=0``) and the ``input features`` (``tweet_input_embeddings`` in our case).

Let us construct the new initial hidden state for a 2 layered RNN.

In [51]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (2)
# second dimension - number of examples/sentences in a batch (4)
# third dimension - number of nodes in hidden layer (50, size of the hidden layer)
h0 = torch.randn(2, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([2, 4, 50])


Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.

In [52]:
"""
forward propagation over the RNN model
"""
print(tweet_input_embeddings.shape)
output, hn = rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

torch.Size([23, 4, 300])


``output`` tensor contains the output features $h_t$ from the last layer of the RNN

In [53]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([23, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [54]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


You can get the last hidden from the last layer by:

In [55]:
hn[-1, :, :]

tensor([[-0.4609,  0.3026,  0.2641, -0.2038,  0.6981, -0.1892,  0.4541,  0.5510,
          0.1726,  0.4292, -0.1855,  0.4309, -0.3680,  0.2882, -0.1988, -0.3617,
          0.2215, -0.6583, -0.1237,  0.3361,  0.1835,  0.0457,  0.2665, -0.1904,
         -0.0842,  0.0119,  0.5735,  0.2388,  0.7321,  0.5106, -0.6631,  0.2452,
         -0.3746, -0.0380,  0.8005, -0.0560,  0.4838,  0.8953, -0.5734,  0.3495,
          0.1620, -0.3643,  0.2641,  0.2496, -0.0880, -0.0731,  0.1830,  0.3835,
         -0.0411,  0.2081],
        [-0.3909,  0.3494,  0.2621, -0.2780,  0.7007, -0.2066,  0.4437,  0.5643,
          0.1590,  0.3533, -0.2263,  0.5208, -0.3536,  0.2857, -0.2617, -0.3738,
          0.2257, -0.6035, -0.0378,  0.3432,  0.1887,  0.0014,  0.3051, -0.2207,
         -0.1585, -0.0614,  0.5037,  0.1902,  0.7197,  0.5308, -0.6304,  0.2652,
         -0.3929,  0.0589,  0.8050, -0.0964,  0.5578,  0.9040, -0.6622,  0.3108,
          0.1165, -0.3817,  0.2763,  0.1697, -0.0720, -0.1678,  0.1378,  0.3047,


Let us compute the final tweet representation:

In [56]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


In [57]:
tweet_output_embeddings

tensor([[-0.4609,  0.3026,  0.2641, -0.2038,  0.6981, -0.1892,  0.4541,  0.5510,
          0.1726,  0.4292, -0.1855,  0.4309, -0.3680,  0.2882, -0.1988, -0.3617,
          0.2215, -0.6583, -0.1237,  0.3361,  0.1835,  0.0457,  0.2665, -0.1904,
         -0.0842,  0.0119,  0.5735,  0.2388,  0.7321,  0.5106, -0.6631,  0.2452,
         -0.3746, -0.0380,  0.8005, -0.0560,  0.4838,  0.8953, -0.5734,  0.3495,
          0.1620, -0.3643,  0.2641,  0.2496, -0.0880, -0.0731,  0.1830,  0.3835,
         -0.0411,  0.2081],
        [-0.3909,  0.3494,  0.2621, -0.2780,  0.7007, -0.2066,  0.4437,  0.5643,
          0.1590,  0.3533, -0.2263,  0.5208, -0.3536,  0.2857, -0.2617, -0.3738,
          0.2257, -0.6035, -0.0378,  0.3432,  0.1887,  0.0014,  0.3051, -0.2207,
         -0.1585, -0.0614,  0.5037,  0.1902,  0.7197,  0.5308, -0.6304,  0.2652,
         -0.3929,  0.0589,  0.8050, -0.0964,  0.5578,  0.9040, -0.6622,  0.3108,
          0.1165, -0.3817,  0.2763,  0.1697, -0.0720, -0.1678,  0.1378,  0.3047,


#### Update END

## RNN for Sentiment Analysis

In this section we will implement RNN for classifying the sentiment of the tweet (same task used in our previous feedforward neural networks tutorial).

We will pick up most of the functions from our feedforward neural networks code:

In [58]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim

# set the seed
manual_seed = 123
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

# hyperparameters
MAX_EPOCHS = 5
LEARNING_RATE = 0.3
NUM_CLASSES = 3
EMBEDDING_SIZE = 300

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Now we can define the full RNN model:

In [59]:
"""
create a model for RNN
"""
class RNNmodel(nn.Module):
  
  def __init__(self, embedding_size, vocab_size, output_size, hidden_size, num_layers, pad_token):
    # In the constructor we define the layers for our model
    super(RNNmodel, self).__init__()
    self.pad_token = pad_token
    # word embedding lookup table
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, sparse=True)
    # core RNN module
    self.rnn_layer = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers) 
    
    ## attention layer
    self.key_a = nn.Linear(hidden_size, hidden_size)
    self.q_a = nn.Linear(hidden_size, hidden_size)
    
    # activation function
    self.activation_fn = nn.ReLU()
    # classification related modules
    self.linear_layer = nn.Linear(hidden_size, output_size) 
    self.softmax_layer = nn.LogSoftmax(dim=0)
  
  def forward(self, x):
    # In the forward function we define the forward propagation logic
    out = self.embedding(x)
    out, _ = self.rnn_layer(out) # since we are not feeding h_0 explicitly, h_0 will be initialized to zeros by default
    # classify based on the hidden representation after RNN processes the last token
    sequence_rep = out[-1]
    
    # self-attention mechanism
    output_keys = self.key_a(out)
    sequence_rep = self.q_a(sequence_rep)
    
    attn_weights = torch.bmm(output_keys.permute(1,0,2), sequence_rep.unsqueeze(2)).squeeze(2).permute(1,0)
    attn_weights = attn_weights.masked_fill(x == self.pad_token, -1e9)
    
    soft_attn_weights = F.softmax(attn_weights, 0).permute(1,0)
    
    new_output_embedding = torch.bmm(out.permute(1, 2, 0), soft_attn_weights.unsqueeze(2)).squeeze(2)
    ##### end attention
    
    out = self.activation_fn(sequence_rep)
    out = self.linear_layer(out)
    out = self.softmax_layer(out) # accepts 2D or more dimensional inputs
    return out

Some additional hyperparameters for RNN

In [60]:
# hyperparameters of RNN
HIDDEN_SIZE = 50
NUM_LAYERS = 2

### Training and Testing Functions.

In [61]:
from sklearn.metrics import accuracy_score
def train(loader):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    return accuracy

Let us define the RNN model.

In [62]:
# define the model
model = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS, TEXT.vocab.stoi[TEXT.pad_token]) 
model.to(device)
# define the loss function (last node of the graph)
criterion = nn.NLLLoss()

**We need to create a new directory 'ckpt/' to store our model checkpoint.**

In [66]:
import os
os.mkdir("./ckpt")

**Let us perform the training. We will save our model and optimizer at end of each epoch.**


You can find more information of saving and loading model [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [67]:
# create an instance of SGD with required hyperparameters
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

# start the training
for epoch in range(MAX_EPOCHS):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc = evaluate(train_iter)
    # compute the validation accuracy
    val_acc = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, val_acc))
    
    # save model, optimizer, and number of epoch to a dictionary
    model_save = {
            'epoch': epoch,  # number of epoch
            'model_state_dict': model.state_dict(), # model parameters 
            'optimizer_state_dict': optimizer.state_dict(), # save optimizer 
            'loss': train_loss # training loss
            }
    
    # use torch.save to store 
    torch.save(model_save, "./ckpt/model_{}.pt".format(epoch))

Epoch [1/5], Loss: 0.3466, Training Accuracy: 0.3300, Validation Accuracy: 0.3042
Epoch [2/5], Loss: 0.3464, Training Accuracy: 0.4350, Validation Accuracy: 0.3652
Epoch [3/5], Loss: 0.3462, Training Accuracy: 0.3768, Validation Accuracy: 0.4072
Epoch [4/5], Loss: 0.3456, Training Accuracy: 0.5047, Validation Accuracy: 0.4237
Epoch [5/5], Loss: 0.3458, Training Accuracy: 0.3950, Validation Accuracy: 0.3472


**Load model checkpoint** 

When we have a trained model checkpint, we can load it using `torch.load()`

In [68]:
# define a new model
model2 = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS, TEXT.vocab.stoi[TEXT.pad_token]) 
# load checkpoint 
checkpoint = torch.load("./ckpt/model_1.pt")
# assign the parameters of checkpoint to this new model
model2.load_state_dict(checkpoint['model_state_dict'])
model2.to(device)

print(model2)

RNNmodel(
  (embedding): Embedding(3343, 300, sparse=True)
  (rnn_layer): RNN(300, 50, num_layers=2)
  (key_a): Linear(in_features=50, out_features=50, bias=True)
  (q_a): Linear(in_features=50, out_features=50, bias=True)
  (activation_fn): ReLU()
  (linear_layer): Linear(in_features=50, out_features=3, bias=True)
  (softmax_layer): LogSoftmax(dim=0)
)


## GRUs

Gated Recurrent Units (GRUs) are a variant of RNNs that use more complex units for activation. They are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies. To learn the theory behind GRUs, we recommend: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 

GRU is defined by ``torch.nn.GRU`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU). Now let us define the GRU module.

In [69]:
"""
define the GRU module
"""
# first input - number of word vector dimensions/embeddings
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (2)
gru_rnn = nn.GRU(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to RNN, GRU module takes two inputs: *the initial hidden state for each element in the batch* (t=0) and the *input features* (``tweet_input_embeddings`` in our case).

Let us feed both the initial hidden state and tweet embeddings to our GRU model.

In [70]:
"""
forward propagation over the GRU model
"""
output, hn = gru_rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contqains the output features $h_t$ from the last layer of the GRU

In [65]:
# output = seq_len, batch, hidden_size (output features from last layer of GRU)
print("output size: ", output.size())

output size:  torch.Size([23, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [71]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


Similar to RNN, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [72]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (5)
# second dimension - number of features in hidden state h_t (20, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


## LSTMs

Long short-term memory (LSTMs) are a variant of RNNs that use more complex units for activation. Similar to the spirit of GRU, they are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies. To learn the theory behind GRUs, we recommend: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 

LSTM is defined by ``torch.nn.LSTM`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM). Now let us define the LSTM module.

In [73]:
"""
define the LSTM module
"""
# first input - number of features in x (300, size of the word embedding)
# second input - number of number of nodes in a hidden layer (50)
# third input - number of recurrent layers (2)
lstm_rnn = nn.LSTM(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Unlike RNN and GRU, LSTM module takes three inputs: the initial hidden state for each element in the batch (t=0), the input features (tweet_input_embeddings in our case) and initial cell state for each element in the batch.

Let us construct the initial cell state (this construction is similar to that of initial hidden state)

In [74]:
"""
cell state at time-step 0 (h_0)
"""
# first dimension - number of LSTM layers (2)
# second dimension - batch_size (# of tweets/examples/sentences)
# third dimension - hidden_size / number of nodes in a hidden layer (50)
c0 = torch.randn(2, 4, 50)

Let us feed the initial hidden state, initial cell state and tweet embeddings to our LSTM model.

In [75]:
"""
forward propagation over the LSTM model
"""
output, (hn, cn) = lstm_rnn(tweet_input_embeddings, None) # h0 and c0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contains the output features $h_t$ from the last layer of the LSTM

In [76]:
# output = seq_len, batch_size, hidden_size (output features from last layer of LSTM)
print("output size: ", output.size())

output size:  torch.Size([23, 4, 50])


``hn`` is a tensor of shape (num_layers, batch, hidden_size) containing the hidden state for t = seq_len

In [77]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


``cn`` is a tensor of shape (num_layers, batch, hidden_size) containing the cell state for t = seq_len.

In [78]:
# c_n = num_layers, batch_size, hidden_size (cell state for t=seq_len or cell state at last timestep)
print("last cell state size: ", hn.size())

last cell state size:  torch.Size([2, 4, 50])


Similar to RNN and GRU, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [79]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])
